In [5]:
%load_ext autoreload
%autoreload 2
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import choice
from collections import defaultdict, Counter
from analysis_utils import get_data
from learning_utils import pickle_load, pickle_save, construct_pipeline, Participant, get_normalized_features,\
                            get_normalized_feature_values, construct_reward_function, reward_levels, reward_type, \
                            construct_repeated_pipeline, create_dir, get_strategy_counts, get_cluster_dict, \
                            get_modified_weights
from sequence_utils import compute_average_click_likelihoods
from generic_mouselab import GenericMouselabEnv
from modified_mouselab import TrialSequence, reward_val, normal_reward_val, constant_reward_val, decreasing_reward_val
from planning_strategies import strategy_dict
from computational_microscope import ComputationalMicroscope
from statsmodels.stats.proportion import proportions_ztest, proportions_chisquare
from scipy.stats import ttest_ind, pearsonr
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy.linalg as LA
from scipy.special import softmax
from IPython.core.display import display, HTML
from experiment_utils import Experiment
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams["axes.grid"] = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
exp_pipelines = pickle_load("data/exp_pipelines.pkl")

In [13]:
cluster_map = pickle_load("data/non_problematic_clusters.pkl")
strategy_space = pickle_load("data/strategy_space.pkl")
num_simulations = 1000000
exp_nums = ["v1.0", "c1.1", "c2.1_dec", "T1.1"]
cluster_scores = defaultdict(lambda: defaultdict(list))
sq_num = np.sqrt(num_simulations)
for exp_num in exp_nums:
    for i, strategy in enumerate(strategy_space):
        #scores = []
        #for sim_num in range(num_simulations):
            #pipeline = exp_pipelines[exp_num]
            #env = GenericMouselabEnv(num_trials=1, pipeline=pipeline)
            #clicks = strategy_dict[strategy](env.present_trial)
            #score = env.present_trial.node_map[0].calculate_max_expected_return()
        scores = pickle_load(f"results/strategy_scores/{exp_num}_{strategy-1}_{num_simulations}.pkl")
        cluster_scores[exp_num][cluster_map[i]] += scores
            #scores.append(score)
        #strategy_scores[exp_num][strategy] = np.mean(scores)
        #ses[exp_num][strategy] = np.std(scores)/sq_num

In [14]:
cluster_mean_scores = defaultdict(lambda: defaultdict(int))
cluster_ses = defaultdict(lambda: defaultdict(int))
for exp_num in exp_nums:
    for c in cluster_scores[exp_num]:
        cluster_mean_scores[exp_num][c] = np.mean(cluster_scores[exp_num][c])
        cluster_ses[exp_num][c] = np.std(cluster_scores[exp_num][c])/len(cluster_scores[exp_num][c])
print(cluster_mean_scores)

defaultdict(<function <lambda> at 0x131c15d90>, {'v1.0': defaultdict(<class 'int'>, {1: 45.3509515, 13: 39.07208152941176, 2: 44.740447333333336, 5: 43.953171, 4: 42.8158216, 6: 45.334842, 10: 32.1941794, 9: 40.9726212, 8: 23.3076252, 7: 7.7588855, 12: 42.95862, 3: 45.333334, 11: 43.989014}), 'c1.1': defaultdict(<class 'int'>, {1: 15.31393875, 13: 11.521002647058824, 2: 14.837128333333334, 5: 13.028325416666666, 4: 13.663711, 6: 15.323355, 10: 7.611349, 9: 8.9220925, 8: 9.755022, 7: 4.32497, 12: 9.298125, 3: 15.314405, 11: 11.976385}), 'c2.1_dec': defaultdict(<class 'int'>, {1: 34.713887, 13: 17.92356882352941, 2: 34.723348, 5: 32.520463666666664, 4: 34.2769104, 6: 34.746704, 10: 6.3481536, 9: 7.2239004, 8: 32.2432862, 7: 12.6630225, 12: 3.582558, 3: 34.72159, 11: 26.34684}), 'T1.1': defaultdict(<class 'int'>, {1: 60.217841998971785, 13: 49.097594323838706, 2: 60.007273986499, 5: 58.87793543462781, 4: 56.88184299472494, 6: 60.20287201866427, 10: 35.75336234825199, 9: 49.45340748548768,

In [15]:
reverse_cluster_map = defaultdict(list)
for i, strategy in enumerate(strategy_space):
    reverse_cluster_map[cluster_map[i]].append(strategy)
for k in sorted(list(reverse_cluster_map.keys())):
    print(k, reverse_cluster_map[k])

1 [1, 8, 9, 13]
2 [3, 10, 82]
3 [72]
4 [5, 36, 37, 54, 79]
5 [4, 6, 7, 11, 12, 31, 45, 46, 47, 59, 60, 84]
6 [15]
7 [28, 30, 34, 66]
8 [22, 23, 32, 33, 53, 64, 65, 69, 70, 80]
9 [21, 40, 41, 43, 55, 56, 57, 58, 63, 67]
10 [16, 26, 27, 29, 44, 50, 61, 73, 78, 89]
11 [75]
12 [51]
13 [2, 14, 17, 18, 24, 39, 42, 48, 49, 62, 71, 74, 76, 85, 86, 87, 88]


In [16]:
cluster_names = ["Goal setting", "Forward planning strategies similar to Breadth First Search", "Middle-out planning", "Forward planning strategies similar to Best First Search", "Local-search strategies", "Maximizing goal-setting with exhaustive backward planning", "Frugal planning strategies", "Myopic planning strategies", "Maximizing goal-setting without backward planning", "Frugal goal-setting", "Strategy that explores immediate outcomes on the paths to the best final outcomes", "Strategy that explores immediate rewards on the paths to the best final outcomes with satisficing", "Other strategies"]
pickle_save(cluster_names, "data/cluster_names.pkl")


In [20]:
exp_labels = ["Increasing variance", "Constant Variance", "Decreasing variance", "Transfer task"]
for exp_label, exp_num in zip(exp_labels, exp_nums):
    print(exp_label)
    sorted_list = sorted(cluster_mean_scores[exp_num].items(), key=operator.itemgetter(1), reverse=True)
    for (c, s) in sorted_list:
        print(cluster_names[c-1], s, "+-", cluster_ses[exp_num][c])
    print("\n")

Increasing variance
Goal setting 45.3509515 +- 3.4122744892945663e-06
Maximizing goal-setting with exhaustive backward planning 45.334842 +- 1.3678780385510836e-05
Middle-out planning 45.333334 +- 1.3664721308700154e-05
Forward planning strategies similar to Breadth First Search 44.740447333333336 +- 4.51182821300765e-06
Strategy that explores immediate outcomes on the paths to the best final outcomes 43.989014 +- 1.2462318697088597e-05
Local-search strategies 43.953171 +- 1.1590125438060254e-06
Strategy that explores immediate rewards on the paths to the best final outcomes with satisficing 42.95862 +- 1.2606465313306499e-05
Forward planning strategies similar to Best First Search 42.8158216 +- 3.349476978377212e-06
Maximizing goal-setting without backward planning 40.9726212 +- 1.3927737447313914e-06
Other strategies 39.07208152941176 +- 1.0845443019193668e-06
Frugal goal-setting 32.1941794 +- 1.9333459710062657e-06
Myopic planning strategies 23.3076252 +- 2.2175691473690854e-06
Frug